In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opendatasets
import opendatasets as od
import os
if not os.path.isdir("vectorragdata"):
  od.download("https://www.kaggle.com/datasets/mohamedmahmod/vectorragdata")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mohamedmahmod
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mohamedmahmod/vectorragdata


100%|██████████| 93.6M/93.6M [00:07<00:00, 13.5MB/s]


In [ ]:
%mv /content/vectorragdata drive/MyDrive

In [18]:
!pip install --upgrade --quiet langchain langchain-community langchain-groq chromadb
!pip install sentence-transformers langchain-chroma

In [19]:
!pip install langchain_core gradio

In [20]:
import sqlite3
import uuid
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the HuggingFace embedding model for Arabic
embedding_function = HuggingFaceEmbeddings(model_name="CAMeL-Lab/bert-base-arabic-camelbert-mix")

# Set the vectorstore path to your Google Drive
vectorstore_path = "/content/drive/MyDrive/vectorragdata"



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
# Load the vectorstore again for querying
vectorstore = Chroma(persist_directory=vectorstore_path, embedding_function=embedding_function)

# Create a retriever for similarity search
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={ 'k': 8})

# Example query
query = "ابي لديه السكري وياخذ الانسولين هل عليه خطوره"
relevant_chunks = retriever.invoke(query)

# Print the relevant chunks (question-answer pairs)
for chunk in relevant_chunks:
    print(chunk.page_content)  # Accessing the page_content attribute


Q: ابي لديه السكري وياخذ الانسولين هل عليه خطوره
A: نهائيا الكروسترول افضل علاج للسكري
Q: ابني عمره 9 سنوات وقبل 2 شهرين تم اكتشاف وتشخيص ان مريض بسكر الدم من الفئه الاولي والحين ياخذ ابر انسلين بعد 3 وجبات الرئيسيه هل يوجد علاج متوفر اغير الابر
A: ممكن استخدام مضخه الانسولين
Q: ابي مريض سكر وياخذ انسولين منذ فتره هل يمكن ان يستغني عنه اذا اتبع نظام غذائي لمرضي السكري وهل صحيح ان العلاج بالاعشاب يشفي من السكر تماما
A: لا يستطيع ترك العلاج والطب بالاعشاب لا يشفي من المرض
Q: السلام عليكم انا عندي الالتهاب لوز مزمن واخذ كل شهر ابره بنسلين طويل المفعول وكذلك مصاب بالسكري ابغي اطعم تطعيم الفولتارين الموسميه هل يتعارض مع البنسلين او ماله علاقه
A: لا علاقه مع السكر
Q: انا مريضه بالسكري عمله فحص سكر التراكمي وجدته 9 هل مشكله صحيه عاديه او معدل السكري زائد
A: نعم هذا شيء غير مسيطر عليه يتم تعديل جرعه الدواء او تبديله او التحول للانسولين بالتعاون مع الطبيب المعالج سلامات
Q: ابي مريض بالسكر اثر صدمه منذ حوالي 10 سنين ومن ذلك الوقت يتعاطي نصف حبه قبرصيهفقط فجرا وسكره معتدل هل هناك امل بالشفاء علما

In [22]:
PROMPT_TEMPLATE = """
أنت طبيب متخصص في الأمراض الباطنية، وأنت هنا للإجابة على الأسئلة المتعلقة بالمجال الطبي.

يرجى التعامل بحرص مع المعلومات التالية التي تخص سؤالك، وتأكد من عدم تقديم أي معلومات غير دقيقة. إذا كانت لديك أي شكوك أو لم يكن لديك معلومات محددة، يُفضل دائمًا زيارة طبيب مختص.

المعلومات المتاحة:

{context}

بناءً على هذه المعلومات، يُرجى الإجابة عن السؤال التالي:
سؤالي هو: {question}

إذا لم تكن متأكدًا من الإجابة، فلا تقم بتقديم أي معلومات خاطئة، وأوصِ بزيارة طبيب.
"""


In [23]:
from langchain_core.prompts import ChatPromptTemplate
# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text,
                                question="ابي لديه السكري وياخذ الانسولين هل عليه خطوره")
print(prompt)

Human: 
أنت طبيب متخصص في الأمراض الباطنية، وأنت هنا للإجابة على الأسئلة المتعلقة بالمجال الطبي.

يرجى التعامل بحرص مع المعلومات التالية التي تخص سؤالك، وتأكد من عدم تقديم أي معلومات غير دقيقة. إذا كانت لديك أي شكوك أو لم يكن لديك معلومات محددة، يُفضل دائمًا زيارة طبيب مختص.

المعلومات المتاحة:

Q: ابي لديه السكري وياخذ الانسولين هل عليه خطوره
A: نهائيا الكروسترول افضل علاج للسكري

---

Q: ابني عمره 9 سنوات وقبل 2 شهرين تم اكتشاف وتشخيص ان مريض بسكر الدم من الفئه الاولي والحين ياخذ ابر انسلين بعد 3 وجبات الرئيسيه هل يوجد علاج متوفر اغير الابر
A: ممكن استخدام مضخه الانسولين

---

Q: ابي مريض سكر وياخذ انسولين منذ فتره هل يمكن ان يستغني عنه اذا اتبع نظام غذائي لمرضي السكري وهل صحيح ان العلاج بالاعشاب يشفي من السكر تماما
A: لا يستطيع ترك العلاج والطب بالاعشاب لا يشفي من المرض

---

Q: السلام عليكم انا عندي الالتهاب لوز مزمن واخذ كل شهر ابره بنسلين طويل المفعول وكذلك مصاب بالسكري ابغي اطعم تطعيم الفولتارين الموسميه هل يتعارض مع البنسلين او ماله علاقه
A: لا علاقه مع السكر

---

Q: انا مريضه

In [24]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-70b-versatile",
               api_key='',
               temperature=0.3,
               max_tokens=1024)


In [27]:
from langchain_core.runnables import RunnablePassthrough
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm
        )
rag_chain.invoke("ابي لديه السكري وياخذ الانسولين هل عليه خطوره").content

'لا، لا يوجد علاج نهائي للسكري، ولكن يمكن السيطرة عليه عن طريق العلاج بالانسولين والتحكم في النظام الغذائي والتمارين الرياضية. لا يوجد دليل على أن الكوليسترول هو علاج نهائي للسكري. من الضروري أن يتابع أبيك مع طبيبه المعالج لضمان السيطرة على مرض السكري بشكل صحيح.'

In [28]:
# Define the function to retrieve an answer using your RAG chain
def answer_question_rag(user_input):
    # This will invoke the RAG chain you have already set up
    return rag_chain.invoke(user_input).content


In [29]:
def get_prompt(user_input):
    # Construct the prompt with the question
    prompt_without_context = f"""
    أنت طبيب متخصص في الأمراض الباطنية، وأنت هنا للإجابة على الأسئلة المتعلقة بالمجال الطبي.

    يرجى التعامل بحرص مع المعلومات التالية التي تخص سؤالك، وتأكد من عدم تقديم أي معلومات غير دقيقة. إذا كانت لديك أي شكوك أو لم يكن لديك معلومات محددة، يُفضل دائمًا زيارة طبيب مختص.
    سؤالي هو: {user_input}

    إذا لم تكن متأكدًا من الإجابة، فلا تقم بتقديم أي معلومات خاطئة، وأوصِ بزيارة طبيب.
    """
    return llm.invoke(prompt_without_context).content  # Return the formatted prompt


In [30]:
def answer_and_prompt(user_input):
    answer = answer_question_rag(user_input)
    prompt = get_prompt(user_input)
    return answer, prompt  # Return both the answer and the prompt


In [32]:
import gradio as gr
interface = gr.Interface(
    fn=answer_and_prompt,  # Use the combined function
    inputs=gr.Textbox(label="أدخل سؤالك الطبي"),
    outputs=[
        gr.Textbox(label="الإجابة"),  # Output for the answer
        gr.Textbox(label="الإجابة من النموذج اللغوي ")  # Output for the prompt with the question
    ]
)

# Launch the Gradio app
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2620056292b01570c7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
